In [1]:
from langfuse import Langfuse
from mtg.utils import load_config


config = load_config("configs/config.yaml")


langfuse = Langfuse()


2024-12-08 11:38:28.716 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 11:38:28.720 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 11:38:28.721 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-12-08 11:38:30.892 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 11:38:30.893 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 11:38:30.893 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

In [ ]:
dataset_name = "expert_questions"
langfuse.create_dataset(
    name=dataset_name,
    description="Important questions from reddit or from app users.",
)

Dataset(id='cm4fh468k001ril4ykmq1n9u8', name='expert_questions', description='Important questions from reddit or from app users.', metadata=None, project_id='cm3ylkdoi01jmo73cji7tla2w', created_at=datetime.datetime(2024, 12, 8, 10, 41, 20, 87000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 12, 8, 10, 41, 20, 87000, tzinfo=datetime.timezone.utc))

In [3]:
import json 
with open("data/expert_knowledge.json", "r", encoding="utf-8") as infile: 
    data = json.load(infile)

data 

[{'question': 'If I control Solphim, Mayhem Dominus and Imodane, the Pyrohammer, and I target an opponent’s creature with a burn spell what happens?',
  'answer': 'The damage of the burn spell gets doubled because of Solphim, Mayhem Dominus. Then Imodane will deal that much damage to every opponent. Solphim, Mayhem Dominus triggered ability acts as a replacement effect for the damage of the burn spell.'},
 {'question': "what happens to Urza's Saga when Blood Moon is played?",
  'answer': "Saga's as a rule are sacrificed as a state based action when the number of counters are equal or greater than the number of chapter abilities (and as long as there isn't a related chapter trigger on the stack). So when Urzas saga loses all of it's abilities due to blood moon or a similar effect it's a saga with zero chapter abilities. Which means it's sacrificed as a state based action."},
 {'question': 'What happens if i controll two creatures and Repercussion and my opponent controlls 2 creatures. W

In [ ]:


for item in data:

    langfuse.create_dataset_item(
        dataset_name=dataset_name,
        input={
            "text": item["question"]
        },
        expected_output={
            "text": item["answer"]
        },
    )

## evaluate dataset 

In [16]:

dataset_name = "expert_questions"

dataset = langfuse.get_dataset(dataset_name)


In [76]:
    
from mtg.agents import nissa
from mtg.tools import (
    CardSearchTool,
    CardNameSearchTool,
    RulesSearchTool,
    UserDeckLookupTool,
)
from mtg.agents import create_chat_agent

model_name = "gpt-4o"

# tools
card_search_tool = CardSearchTool(
    url=config.dataservice_settings.host,
    threshold=config.dataservice_settings.card_search_threshold,
    number_of_cards=config.dataservice_settings.card_search_number_of_cards,
)

card_name_search_tool = CardNameSearchTool(
    url=config.dataservice_settings.host,
)

rules_search_tool = RulesSearchTool(
    url=config.dataservice_settings.host,
    threshold=config.dataservice_settings.rules_search_threshold,
)
deck_tool = UserDeckLookupTool()

# agents
agent = create_chat_agent(
    system_message=nissa.SYSTEM_MESSAGE,
    prompt=nissa.PROMPT,
    tools=[
        deck_tool,
        card_search_tool,
        card_name_search_tool,
        rules_search_tool,
    ],
    memory=None,
    model_name=model_name,
)

In [67]:
# evaluation agent 
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
)
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser


from dotenv import load_dotenv
from pydantic import BaseModel, Field
load_dotenv()

EVALUATION_PROMPT = """
Evaluate the correctness of the generation on a continuous scale from 0 to 1. A generation can be considered correct (Score: 1) if it includes all the key facts from the ground truth and if every fact presented in the generation is factually supported by the ground truth and is consistent with official magic the gathering rules.

Example:

Query: Can you use Lightning Bolt to destroy a creature with Indestructible?

Generation: Yes, Lightning Bolt can destroy a creature with Indestructible if it deals enough damage. People often misunderstand Indestructible, but it simply means the creature cannot die from damage alone—it doesn’t stop spells like Lightning Bolt from affecting it. In fact, casting Lightning Bolt is one of the best ways to deal with Indestructible creatures because it gets around their ability and sends them straight to the graveyard. The Indestructible keyword is often overhyped, and many players don’t realize how easy it is to bypass with direct damage.

Ground truth: No, Lightning Bolt cannot destroy a creature with Indestructible. The Indestructible keyword means that the creature cannot be destroyed by damage or effects that say “destroy.” Damage can still be dealt to it, and it will accumulate, but it will not result in the creature being destroyed. However, the creature can still be removed from the battlefield through other means, such as being exiled or returned to its owner’s hand.

Score: 0.2
Reasoning: While the generation correctly mentions that Lightning Bolt can deal damage to an Indestructible creature, it falsely claims that this would destroy the creature. The explanation includes a misunderstanding of the Indestructible keyword and misleads readers by stating that Lightning Bolt is a way to bypass it entirely. The response significantly deviates from the accurate explanation in the ground truth.


Input:
Query: {query}
Generation: {generation}
Ground truth: {ground_truth}


Think step by step.
Use the Evaluation Report to give your answer!
"""

class EvaluationReport(BaseModel):
    """Evaluation Report."""

    reasoning: str = Field(description="reasoning why the generation includes all the key facts from the ground truth")
    score: float = Field(description="the correctness of the generation on a continuous scale from 0 to 1")


llm = ChatOpenAI(
    model=model_name,
    temperature=0.001,
    n=1,
    streaming=False,
)

prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful assistant that evaluates a magic the gathering rules advice program."), ("user", EVALUATION_PROMPT)]
)

llm_with_functions = llm.bind_tools([EvaluationReport])


evaluation_chain = prompt | llm_with_functions | JsonOutputKeyToolsParser(key_name="EvaluationReport")



In [56]:
llm_with_functions.kwargs["tools"]
evaluation_chain


PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nEvaluate the correctness of the generation on a continuous scale from 0 to 1. A generation can be considered correct (Score: 1) if it includes all the key facts from the ground truth and if every fact presented in the generation is factually supported by the ground truth and is consistent with official magic the gathering rules.\n\nExample:\n\nQuery: Can you use Lightning Bolt to destroy a creature with Indestructible?\n\nGeneration: Yes, Lightning Bolt can destroy a creature with Indestructible if it deals enough damage. People often misunderstand Indestructible, but it simply means the creature cannot die from damage alone—it doesn’t stop spells like Lightning Bolt from affecting it. In fact, casting Lightning Bolt is one of the best ways to deal with Indestructible creatures because it gets around their ability and sends them straight to the graveyard. The Indestructible keyword is often overhyped, 

In [77]:
from langfuse.callback import CallbackHandler

langfuse_handler = CallbackHandler()

for item in dataset.items:
    # Make sure your application function is decorated with @observe decorator to automatically link the trace
    with item.observe(
        run_name="Nissa gpt-4o",
        run_description="Evaluating Nissa on Expert questions",
        run_metadata={"model": model_name},
    ) as trace_id:
        # run your @observe() decorated application on the dataset item input
        answer = agent.invoke(
            {"human_input":item.input["text"], "user_decks": "user did not upload any decks!", "history": []}, 
            config={
                "callbacks": [langfuse_handler],
                "run_id": trace_id,
                "run_name": "Nissa"}
        )

        evaluation_response = evaluation_chain.invoke({"query": item.input["text"], "generation":answer["output"], "ground_truth":item.expected_output["text"]})
        evaluation_response = evaluation_response[0]

        # optionally, evaluate the output to compare different runs more easily
        langfuse.score(
            trace_id=trace_id,
            name="EvaluationReport",
            value=float(evaluation_response["score"]),
            comment=evaluation_response["reasoning"],
        )
 
# Flush the langfuse client to ensure all data is sent to the server at the end of the experiment run
langfuse.flush()

2024-12-08 12:56:14,793 - mtg.tools.card_search_tool - INFO - Triggering Card Name Search with : 'Teysa Karlov'  (card_search_tool.py:174)
2024-12-08 12:56:16,845 - mtg.tools.card_search_tool - INFO - received cards [Teysa Karlov] from card search tool (card_search_tool.py:146)
2024-12-08 12:56:17,661 - mtg.tools.card_search_tool - INFO - Triggering Card Name Search with : 'Ratadrabik of Urborg'  (card_search_tool.py:174)
2024-12-08 12:56:19,721 - mtg.tools.card_search_tool - INFO - received cards [Ratadrabik of Urborg] from card search tool (card_search_tool.py:146)
2024-12-08 12:56:20,571 - mtg.tools.rules_search_tool - INFO - Triggering Rules Search with query: Teysa Karlov and Ratadrabik of Urborg interaction (rules_search_tool.py:53)
2024-12-08 12:56:22,754 - mtg.tools.rules_search_tool - INFO - received 10 documents from rules search tool (rules_search_tool.py:93)
2024-12-08 12:56:28,796 - mtg.tools.card_search_tool - INFO - Triggering Card Name Search with : 'Pantlaza, Sun-Favor

In [70]:
evaluation_response

[{'reasoning': "The generation correctly identifies that Ratadrabik of Urborg's ability triggers when Teysa Karlov dies, creating a token copy of Teysa. However, it incorrectly states that Teysa Karlov's ability does not apply to herself when she dies. According to the ground truth, Teysa Karlov's ability should trigger and cause Ratadrabik's ability to create two tokens instead of one. The generation fails to recognize the doubling effect of Teysa's ability on Ratadrabik's token creation, which is a key fact in the ground truth. Therefore, the generation is partially correct but misses a crucial interaction between the abilities of Teysa Karlov and Ratadrabik of Urborg.",
  'score': 0.5}]